In [8]:
import os
import openai

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

openai_api_key = os.getenv("OPENAI_API_KEY")

In [9]:
openai.api_key = openai_api_key
client = openai.Client()

In [44]:
mensagens = ""

mensagens = [{'role' : 'user', 'content' : 'O que é uma maça em 5 palavras?'}]

resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0
)

In [45]:
print(resposta.choices[0].message.content)

Fruta redonda e saborosa.


In [46]:
mensagens.append({'role': 'assistant', 'content': resposta.choices[0].message.content})

In [47]:
mensagens.append({'role': 'user', 'content': 'E qual é a sua cor?'})

In [48]:
mensagens

[{'role': 'user', 'content': 'O que é uma maça em 5 palavras?'},
 {'role': 'assistant', 'content': 'Fruta redonda e saborosa.'},
 {'role': 'user', 'content': 'E qual é a sua cor?'}]

In [49]:
resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)

Vermelha, verde ou amarela.


In [51]:
## Criando Método

## CRIANDO UM MÉTODO

In [55]:
def geracao_texto(mensagens, model='gpt-3.5-turbo-0125', max_tokens=1000, temperature=0):
    resposta = client.chat.completions.create(
        messages=mensagens,
        model=model,
        max_tokens=max_tokens,
        temperature=temperature
    )

    print(resposta.choices[0].message.content)

    # mensagens.append({'role': 'assistant', 'content': resposta.choices[0].message.content}) # Tem que dizer na mão se é user ou assistant
    mensagens.append(resposta.choices[0].message.model_dump(exclude_none=True)) # Já faz essa descrição sozinho

    return mensagens

In [56]:
mensagens = [{'role' : 'user', 'content' : 'O que é uma maça em 5 palavras?'}]
mensagens = geracao_texto(mensagens)

Fruta redonda e saborosa.


In [57]:
mensagens.append({'role': 'user', 'content': 'E qual é a sua cor?'})
mensagens = geracao_texto(mensagens)

Vermelha, verde ou amarela.


## Explorando max_tokens e temperature

In [58]:
mensagens = [{'role' : 'user', 'content' : 'O que é uma maça em 5 palavras?'}]
mensagens = geracao_texto(mensagens, max_tokens=2)

Fruta


In [59]:
mensagens = [{'role' : 'user', 'content' : 'O que é uma maça em 5 palavras?'}]
mensagens = geracao_texto(mensagens, max_tokens=4)

Fruta redonda


In [74]:
mensagens = [{'role' : 'user', 'content' : 'O que é uma maça em no maximo 10 palavras?'}]
mensagens = geracao_texto(mensagens, max_tokens=20, temperature=0)

Fruta redonda, suculenta e crocante, geralmente de cor vermelha


In [75]:
mensagens = [{'role' : 'user', 'content' : 'O que é uma maça em no maximo 10 palavras?'}]
mensagens = geracao_texto(mensagens, max_tokens=20, temperature=1)

Uma fruta redonda, suculenta e levemente ácida.


In [77]:
mensagens = [{'role' : 'user', 'content' : 'O que é uma maça em no maximo 10 palavras em portugues?'}]
mensagens = geracao_texto(mensagens, max_tokens=20, temperature=2)

Um tipo de fruta, sino belum extern sesational frustrating_traffic forever.handler.getPage refl接amberasionally
